In [ ]:
from extraction import extract_articles_from_google
from extraction import extract_data_from_page
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By

In [ ]:
pages_file_name = "the_clinic_articles.json"
content_file_name = "the_clinic_articles_with_content.json"

In [ ]:
extract_articles_from_google(
	website="theclinic.cl",
	newscast="The Clinic",
    keywords="Estallido Social",
    output=pages_file_name,
    pages=2,
    )

In [ ]:
def get_content(body: WebElement):
	content = ""
	for element in body.find_element(By.CLASS_NAME, "the-content").find_elements(By.TAG_NAME, "p"):
		if "Volver al Home" in element.text or "También puedes leer" in element.text:
			continue
		content += element.text.strip() + "\n"
	return content

def get_author(body: WebElement):
	try:
		return body.find_element(By.CLASS_NAME, "autor").find_element(By.TAG_NAME, "a").text.strip()
	except:
		return "not found"

def get_description(body: WebElement):
	return "not found"


extract_data_from_page(
	input_file=pages_file_name, 
	output_file=content_file_name,
	get_author=get_author,
	get_description=get_description,
	get_content=get_content,
 	limit_of_pages=1
)